<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Bubble_Dew_Temperature_NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [2]:
from dotmap import DotMap
from dotmap import DotMap
import jax
import jax.numpy as jnp
from jax.config import config
from jax.experimental.host_callback import id_print
config.update("jax_enable_x64", True) #JAX default is 32 bit - enable 64 bit - double precision

from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

from tools.tree_array_transform2 import VSC, Comp, Range, RangeArray
import tools.che as che
R=8.314 # J/(mol K)

In [3]:
p = che.Props(['Ethanol', 'Water'])

In [4]:
def bubble_T(c):
    return jnp.sum(c.x * p.NRTL_gamma(c.x, c.T)* p.Pvap(c.T))-c.P

In [5]:
c=DotMap()
c.x = jnp.array([0.5,0.5])
c.T = Range(300., 273., 400.)
c.P = 101325.
c.T = Range(300., 273.15, 400.)
vsc = VSC(c, bubble_T)
vsc.solve(verbosity=0)
print(vsc.v.T)

351.12268612063156


In [17]:
def dew_T(c):
  return c.x * p.NRTL_gamma(c.x,c.T)* p.Pvap(c.T) - c.y*c.P

c=DotMap()
c.y = jnp.array([0.5,0.5])
c.P = 101325.
c.T = Range(300., 273.15, 400.)
c.x = Comp(jnp.array([0.5,0.5]))
vsc = VSC(c, dew_T)
vsc.solve(verbosity=0)
print(vsc.v.T)

359.29972101210564


In [7]:
c = DotMap()
c.T = Range(300, 273.15, 400.)
P=101325
molfracs = jnp.linspace(0.,1.,21)

def solve_bubble_T(x1, P):
  c.x = jnp.array([x1, 1-x1])
  c.P = P
  vsc = VSC(c, bubble_T)
  vsc.solve(verbosity=0)
  return (vsc.v.T)

bubble_T_array = jnp.array([solve_bubble_T(x1, P) for x1 in molfracs])

In [8]:
bubble_T_array

DeviceArray([373.14781771, 367.81888961, 363.81033247, 360.70745809,
             358.26221125, 356.31517619, 354.75774653, 353.51213146,
             352.52003665, 351.73589877, 351.12268612, 350.64926994,
             350.28888755, 350.01854559, 349.81950859, 349.67943391,
             349.59750875, 349.59573202, 349.7439828 , 350.21912038,
             351.45840044], dtype=float64)

In [13]:
def solve_dew_T(y1, P):
  c.y = jnp.array([y1, 1-y1])
  c.P = P
  c.x = Comp(jnp.array([0.5,0.5]))
  vsc = VSC(c, dew_T)
  vsc.solve(verbosity=0)
  return (vsc.v.T)

dew_T_array = jnp.array([solve_dew_T(y1, P) for y1 in molfracs])

In [14]:
dew_T_array

DeviceArray([373.14781771, 371.98464787, 370.78094094, 369.53375536,
             368.23989663, 366.89592672, 365.49820485, 364.0429884 ,
             362.52664846, 360.94610755, 359.29972101, 357.58908236,
             355.82287177, 354.02560097, 352.25942654, 350.68524888,
             349.70855532, 349.64877847, 350.08580436, 350.72516738,
             351.45840044], dtype=float64)

In [19]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=molfracs, y=bubble_T_array, name='Liq', mode='lines')
fig.add_scatter(x=molfracs, y=dew_T_array, name='Vap', mode='lines')
fig.update_xaxes(showline=True, linewidth=1, title_text='x1, y1')
fig.update_yaxes(showline=True, linewidth=1, title_text='Temperature (K)')
fig.update_layout(width=800,height=600, title_text=f'EtOH / Water ({P:.0f} Pa)', title_x=0.5)